In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import ipywidgets as widgets
from ipywidgets import interact

In [2]:
try:
    web_data = requests.get('https://finance.yahoo.com/quote/AAPL/sustainability').text
    print('Success')
except:
    print('Could not get URL')

Success


In [3]:
soup = BeautifulSoup(web_data, "html.parser")

In [4]:
# Scraping the overall ESG score
esg_score = soup.find('div', {'class':'Fz(36px) Fw(600) D(ib) Mend(5px)'})
print(esg_score)

<div class="Fz(36px) Fw(600) D(ib) Mend(5px)" data-reactid="20">17</div>


In [5]:
data_point = esg_score.text
print(data_point)

17


In [6]:
# Scraping the specific ESG scores
elements = []
scores = soup.find_all('div', {'class':'D(ib) Fz(23px) smartphone_Fz(22px) Fw(600)'})
for score in scores:
    elements.append(score.text)

In [7]:
print(elements)

['0.1', '7.7', '8.9']


In [8]:
# Scraping controversy level data
controversy_score = soup.find('div', {'class':'D(ib) Fz(36px) Fw(500)'})
ctr_data = controversy_score.text
print(ctr_data)

3


In [9]:
df = pd.DataFrame({'Total ESG Score': data_point, 
                 'Environment': elements[0], 
                 'Social': elements[1], 
                 'Governance': elements[2], 
                 'Controversy Score': ctr_data}, 
                 index = ['AAPL'])
df

,Total ESG Score,Environment,Social,Governance,Controversy Score
AAPL,17,0.1,7.7,8.9,3


In [10]:
# create a function that gathers all of the tickers in the S&P 500
# def get_tickers():
wiki_page = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies').text
sp_500 = pd.read_html(wiki_page)
ticker_df = sp_500[0]
tickers = ticker_df['Symbol']
# return ticker_options

In [11]:
type(tickers)

pandas.core.series.Series

In [12]:
tickers.head(20)

0      MMM
1      ABT
2     ABBV
3     ABMD
4      ACN
5     ATVI
6     ADBE
7      AMD
8      AAP
9      AES
10     AFL
11       A
12     APD
13    AKAM
14     ALK
15     ALB
16     ARE
17    ALXN
18    ALGN
19    ALLE
Name: Symbol, dtype: object

In [13]:
# Gather all ESG scores of S&P 500 firms
def yahoo_finance_esg(ticker):
    elements = []        
    web_data = requests.get('https://finance.yahoo.com/quote/'+ticker+'/sustainability?p='+ticker).text        
    soup = BeautifulSoup(web_data, "html.parser")
    esg_score = soup.find('div', {'class':'Fz(36px) Fw(600) D(ib) Mend(5px)'})
    try:
        data_point = esg_score.text # if data_point == None:
    except:
        return None
    controversy_score = soup.find('div', {'class':'D(ib) Fz(36px) Fw(500)'})
    ctr_data = controversy_score.text
    scores = soup.find_all('div', {'class':'D(ib) Fz(23px) smartphone_Fz(22px) Fw(600)'})
    for score in scores:
        elements.append(score.text)
    
    df_esg = pd.DataFrame({'Total ESG Score': data_point, 
                   'Environment': elements[0], 
                   'Social': elements[1], 
                   'Governance': elements[2], 
                   'Controversy Score': ctr_data}, 
                   index = [ticker])
    return df_esg

In [14]:
full_data = pd.DataFrame({'Total ESG Score': [], 
                   'Environment': [], 
                   'Social': [], 
                   'Governance':[], 
                   'Controversy Score': []})

for i in tickers:
    individual_record = yahoo_finance_esg(i)
    if individual_record is not None:
        print(i + " retrieved") # Progress checker
        full_data = full_data.append(individual_record)
    else:
        print("No data for " + i) # Progress checker
print(full_data)

MMM retrieved
ABT retrieved
ABBV retrieved
No data for ABMD
ACN retrieved
ATVI retrieved
ADBE retrieved
No data for AMD
AAP retrieved
AES retrieved
AFL retrieved
A retrieved
APD retrieved
AKAM retrieved
No data for ALK
ALB retrieved
ARE retrieved
ALXN retrieved
No data for ALGN
ALLE retrieved
LNT retrieved
ALL retrieved
GOOGL retrieved
No data for GOOG
MO retrieved
AMZN retrieved
No data for AMCR
AEE retrieved
AAL retrieved
AEP retrieved
AXP retrieved
AIG retrieved
AMT retrieved
AWK retrieved
AMP retrieved
ABC retrieved
AME retrieved
AMGN retrieved
APH retrieved
ADI retrieved
ANSS retrieved
ANTM retrieved
No data for AON
AOS retrieved
APA retrieved
AAPL retrieved
AMAT retrieved
APTV retrieved
ADM retrieved
ANET retrieved
AJG retrieved
AIZ retrieved
T retrieved
ATO retrieved
ADSK retrieved
ADP retrieved
AZO retrieved
AVB retrieved
AVY retrieved
No data for BKR
BLL retrieved
BAC retrieved
BK retrieved
BAX retrieved
BDX retrieved
No data for BRK.B
BBY retrieved
BIO retrieved
BIIB retrieve

In [15]:
# Save the DataFrame to a csv file
full_data.to_csv('esg_score.csv')

In [82]:
# Gather all stock data of S&P 500 firms
def yahoo_finance_stock(ticker):
    all_vals = []
    # scraping profile information
    profile_web = requests.get('https://finance.yahoo.com/quote/'+ticker+'/profile?p='+ticker).text
    soup = BeautifulSoup(profile_web, "html.parser")
    sector = soup.find('span', {'class':'Fw(600)'}) 
    #print(sector)
    try:
        sector_info = sector.text # if data_point == None:
    except:
        return None
    # srapping financial stats
    statistics_web = requests.get('https://finance.yahoo.com/quote/'+ticker+'/key-statistics?p='+ticker).text  
    soup = BeautifulSoup(statistics_web, "html.parser")
    stats_table = soup.find_all('td', {'class':'Fw(500) Ta(end) Pstart(10px) Miw(60px)'})
    print(stats_table)
    for table_val in stats_table:
        all_vals.append(table_val.text)
    deps = all_vals[3]
    q_growth = all_vals[5]
    #scores = soup.find_all('div', {'class':'D(ib) Fz(23px) smartphone_Fz(22px) Fw(600)'})
    #for score in scores:
        #elements.append(score.text)
    
    df_stats = pd.DataFrame({'Sector': sector_info, 
                           'Diluted EPS': deps, 
                           'Quarterly Growth': q_growth}, 
                          index = [ticker])
    return df_stats

In [83]:
stock_stats = pd.DataFrame({'Sector':[],
                            'Diluted EPS': [], 
                            'Quarterly Growth': []})

for i in tickers[0:3]:
    individual_record = yahoo_finance_stock(i)
    if individual_record is not None:
        print(i + " retrieved") # Progress checker
        stock_stats = stock_stats.append(individual_record)
    else:
        print("No data for " + i) # Progress checker
print(stock_stats)

[<td class="Fw(500) Ta(end) Pstart(10px) Miw(60px)" data-reactid="21">107.06B</td>, <td class="Fw(500) Ta(end) Pstart(10px) Miw(60px)" data-reactid="28">119.72B</td>, <td class="Fw(500) Ta(end) Pstart(10px) Miw(60px)" data-reactid="35">19.99</td>, <td class="Fw(500) Ta(end) Pstart(10px) Miw(60px)" data-reactid="42">17.78</td>, <td class="Fw(500) Ta(end) Pstart(10px) Miw(60px)" data-reactid="49">2.52</td>, <td class="Fw(500) Ta(end) Pstart(10px) Miw(60px)" data-reactid="56">3.33</td>, <td class="Fw(500) Ta(end) Pstart(10px) Miw(60px)" data-reactid="63">8.30</td>, <td class="Fw(500) Ta(end) Pstart(10px) Miw(60px)" data-reactid="70">3.72</td>, <td class="Fw(500) Ta(end) Pstart(10px) Miw(60px)" data-reactid="77">13.33</td>, <td class="Fw(500) Ta(end) Pstart(10px) Miw(60px)" data-reactid="95">0.95</td>, <td class="Fw(500) Ta(end) Pstart(10px) Miw(60px)" data-reactid="102">23.05%</td>, <td class="Fw(500) Ta(end) Pstart(10px) Miw(60px)" data-reactid="109">41.37%</td>, <td class="Fw(500) Ta(en

[<td class="Fw(500) Ta(end) Pstart(10px) Miw(60px)" data-reactid="21">190.38B</td>, <td class="Fw(500) Ta(end) Pstart(10px) Miw(60px)" data-reactid="28">267.21B</td>, <td class="Fw(500) Ta(end) Pstart(10px) Miw(60px)" data-reactid="35">39.64</td>, <td class="Fw(500) Ta(end) Pstart(10px) Miw(60px)" data-reactid="42">7.81</td>, <td class="Fw(500) Ta(end) Pstart(10px) Miw(60px)" data-reactid="49">1.90</td>, <td class="Fw(500) Ta(end) Pstart(10px) Miw(60px)" data-reactid="56">4.16</td>, <td class="Fw(500) Ta(end) Pstart(10px) Miw(60px)" data-reactid="63">14.55</td>, <td class="Fw(500) Ta(end) Pstart(10px) Miw(60px)" data-reactid="70">5.83</td>, <td class="Fw(500) Ta(end) Pstart(10px) Miw(60px)" data-reactid="77">12.05</td>, <td class="Fw(500) Ta(end) Pstart(10px) Miw(60px)" data-reactid="95">0.81</td>, <td class="Fw(500) Ta(end) Pstart(10px) Miw(60px)" data-reactid="102">25.86%</td>, <td class="Fw(500) Ta(end) Pstart(10px) Miw(60px)" data-reactid="109">41.37%</td>, <td class="Fw(500) Ta(en

In [70]:
stock_stats

,Sector,Diluted EPS,Quarterly Growth


In [ ]:
# Save the DataFrame to a csv file
stock_stats.to_csv('stock_stats.csv')